# Example 3: 複数アクティビティ比較

このノートブックは、複数のアクティビティを比較してパフォーマンスの変化を分析する例を示します。

## ユースケース

**ユーザーの質問**: 「過去5回の10kmランの心拍ゾーン推移を比較したい」

## アーキテクチャフロー

1. **プロファイル取得**: `profile()` で各アクティビティの要約統計を取得
2. **データエクスポート**: `export()` で複数アクティビティの心拍データを取得
3. **時間軸正規化**: Pythonで各アクティビティを共通の時間軸に整列
4. **重ね合わせグラフ**: 5アクティビティの心拍推移を可視化
5. **差異分析**: 統計サマリー + 傾向分析
6. **解釈**: LLMがパフォーマンス変化を解説

## トークン削減のポイント

- プロファイル: 各アクティビティ約500バイト × 5 = 2.5KB（約625トークン）
- エクスポート: ハンドル × 5 = 約125トークン
- 要約: 統計 + 傾向 = 約150トークン
- 合計: 約900トークン（生データ直接読み込みなら約15,000トークン → 94%削減）

## セットアップ

In [ ]:
import sys
from datetime import datetime, timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl

# プロジェクトルートをPATHに追加
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

from tools.utils.llm_safe_data import safe_json_output, safe_load_export

## ステップ1: アクティビティプロファイルの取得（シミュレート）

実際の環境では、以下のように各アクティビティのプロファイルを取得します：

```python
profiles = []
for activity_id in [12345, 12346, 12347, 12348, 12349]:
    profile = mcp__garmin_db__profile(
        table_or_query=f"SELECT * FROM splits WHERE activity_id = {activity_id}"
    )
    profiles.append(profile)
```

In [ ]:
# プロファイルのモック（5つのアクティビティ）
base_date = datetime(2025, 1, 1)
activity_ids = [12345, 12346, 12347, 12348, 12349]

profiles = []
for i, activity_id in enumerate(activity_ids):
    profile = {
        'activity_id': activity_id,
        'date': (base_date + timedelta(weeks=i)).strftime('%Y-%m-%d'),
        'distance_km': 10.0 + np.random.uniform(-0.2, 0.2),
        'duration_seconds': 2700 + i * 60,  # 改善傾向（速くなっている）
        'avg_pace_sec_per_km': 270 + i * 6,  # 改善傾向（遅くなっている → 疲労？）
        'avg_heart_rate': 150 - i * 2,  # 改善傾向（心拍が下がっている → 効率化）
        'hr_zone_distribution': {
            'zone_1': 5 + i * 2,
            'zone_2': 15 + i * 3,
            'zone_3': 40 - i * 2,
            'zone_4': 30 - i * 3,
            'zone_5': 10
        }
    }
    profiles.append(profile)

print("アクティビティプロファイル:")
for profile in profiles:
    print(f"\n  Activity {profile['activity_id']} ({profile['date']}):")
    print(f"    距離: {profile['distance_km']:.2f}km")
    print(f"    時間: {profile['duration_seconds']//60}:{profile['duration_seconds']%60:02d}")
    print(f"    平均ペース: {profile['avg_pace_sec_per_km']//60}:{profile['avg_pace_sec_per_km']%60:02d}/km")
    print(f"    平均心拍: {profile['avg_heart_rate']:.0f} bpm")

print("\nトークンコスト: 約625トークン（5アクティビティ × 125トークン）")

## ステップ2: 心拍データのエクスポート（シミュレート）

実際の環境では、各アクティビティの心拍データをエクスポートします：

```python
export_results = []
for activity_id in activity_ids:
    export_result = mcp__garmin_db__export(
        query=f"""
            SELECT timestamp, heart_rate
            FROM time_series_metrics
            WHERE activity_id = {activity_id}
        """,
        format="parquet"
    )
    export_results.append(export_result)
```

In [ ]:
# モックデータ: 各アクティビティの心拍データ
np.random.seed(42)
export_results = []

for i, (activity_id, profile) in enumerate(zip(activity_ids, profiles, strict=False)):
    duration = profile['duration_seconds']
    timestamps = np.arange(0, duration)

    # 心拍の時系列: ウォームアップ → 安定 → 上昇（疲労）
    warmup = np.linspace(120, 150, duration // 10)  # 最初の10%でウォームアップ
    steady = np.full(duration * 7 // 10, 150 - i * 2) + np.random.normal(0, 3, duration * 7 // 10)
    fatigue = np.linspace(150 - i * 2, 160 - i * 2, duration - len(warmup) - len(steady))  # 最後の20%で疲労

    heart_rate = np.concatenate([warmup, steady, fatigue])

    df = pl.DataFrame({
        'timestamp': timestamps,
        'heart_rate': heart_rate
    })

    # Parquetとして保存
    export_path = Path(f'/tmp/export_activity_{activity_id}.parquet')
    df.write_parquet(export_path)

    export_result = {
        'activity_id': activity_id,
        'handle': str(export_path),
        'rows': len(df),
        'size_mb': export_path.stat().st_size / (1024 * 1024),
        'columns': ['timestamp', 'heart_rate']
    }
    export_results.append(export_result)

print("エクスポート結果:")
for result in export_results:
    print(f"  Activity {result['activity_id']}: {result['rows']} 行")

print("\nトークンコスト: 約125トークン（5アクティビティ × 25トークン）")

## ステップ3: データをロードして時間軸を正規化

各アクティビティの時間軸を0-100%に正規化して比較可能にします。

In [ ]:
# データをロードして正規化
activities_data = []

for export_result, profile in zip(export_results, profiles, strict=False):
    df = safe_load_export(export_result['handle'], max_rows=10000)

    # 時間軸を0-100%に正規化
    df = df.with_columns([
        (pl.col('timestamp') / pl.col('timestamp').max() * 100).alias('progress_pct')
    ])

    activities_data.append({
        'activity_id': export_result['activity_id'],
        'date': profile['date'],
        'df': df
    })

print(f"✓ {len(activities_data)} アクティビティのデータをロード・正規化")

## ステップ4: 重ね合わせグラフの生成

In [ ]:
# 重ね合わせグラフ
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左: 正規化時間軸での心拍推移
colors = plt.cm.viridis(np.linspace(0, 1, len(activities_data)))
for i, activity in enumerate(activities_data):
    df_pandas = activity['df'].to_pandas()
    ax1.plot(df_pandas['progress_pct'], df_pandas['heart_rate'],
             label=f"{activity['date']}", color=colors[i], alpha=0.7, linewidth=2)

ax1.set_xlabel('進捗 (%)')
ax1.set_ylabel('心拍 (bpm)')
ax1.set_title('心拍ゾーン推移の比較（正規化時間軸）')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右: 平均心拍の推移（トレンド）
avg_hrs = [profile['avg_heart_rate'] for profile in profiles]
dates = [profile['date'] for profile in profiles]

ax2.plot(range(len(avg_hrs)), avg_hrs, 'o-', color='blue', linewidth=2, markersize=8)
ax2.set_xticks(range(len(dates)))
ax2.set_xticklabels(dates, rotation=45, ha='right')
ax2.set_ylabel('平均心拍 (bpm)')
ax2.set_title('平均心拍のトレンド')
ax2.grid(True, alpha=0.3)

# トレンドライン
z = np.polyfit(range(len(avg_hrs)), avg_hrs, 1)
p = np.poly1d(z)
ax2.plot(range(len(avg_hrs)), p(range(len(avg_hrs))), "r--", alpha=0.7,
         label=f'トレンド (傾き: {z[0]:.1f} bpm/週)')
ax2.legend()

plt.tight_layout()
plot_path = Path('/tmp/multi_activity_comparison_plot.png')
plt.savefig(plot_path, dpi=100, bbox_inches='tight')
plt.show()

print(f"✓ グラフを保存: {plot_path}")

## ステップ5: 統計サマリーの計算

In [ ]:
# 統計サマリー
summary = {
    'num_activities': len(profiles),
    'date_range': {
        'start': profiles[0]['date'],
        'end': profiles[-1]['date']
    },
    'performance_trend': {
        'avg_heart_rate': {
            'first': float(profiles[0]['avg_heart_rate']),
            'last': float(profiles[-1]['avg_heart_rate']),
            'change': float(profiles[-1]['avg_heart_rate'] - profiles[0]['avg_heart_rate']),
            'change_pct': float((profiles[-1]['avg_heart_rate'] - profiles[0]['avg_heart_rate']) / profiles[0]['avg_heart_rate'] * 100)
        },
        'avg_pace': {
            'first': float(profiles[0]['avg_pace_sec_per_km']),
            'last': float(profiles[-1]['avg_pace_sec_per_km']),
            'change': float(profiles[-1]['avg_pace_sec_per_km'] - profiles[0]['avg_pace_sec_per_km']),
            'change_pct': float((profiles[-1]['avg_pace_sec_per_km'] - profiles[0]['avg_pace_sec_per_km']) / profiles[0]['avg_pace_sec_per_km'] * 100)
        }
    },
    'hr_zone_trend': {
        'zone_3_change': profiles[-1]['hr_zone_distribution']['zone_3'] - profiles[0]['hr_zone_distribution']['zone_3'],
        'zone_4_change': profiles[-1]['hr_zone_distribution']['zone_4'] - profiles[0]['hr_zone_distribution']['zone_4']
    },
    'plot_path': str(plot_path)
}

# 要約JSONを出力（1KB制限）
summary_json = safe_json_output(summary)
print("要約データ（LLMに返却）:")
print(summary_json)
print(f"\nJSON サイズ: {len(summary_json.encode('utf-8'))} バイト")
print("トークンコスト: 約150トークン")

## ステップ6: LLMによるパフォーマンス変化の解説（シミュレート）

In [ ]:
# LLMの解釈例
interpretation = f"""
**過去5回の10kmランの比較分析:**

**期間**: {summary['date_range']['start']} ~ {summary['date_range']['end']} (5週間)

**パフォーマンストレンド:**

1. **心拍効率の改善（ポジティブ）**
   - 平均心拍: {summary['performance_trend']['avg_heart_rate']['first']:.0f} → {summary['performance_trend']['avg_heart_rate']['last']:.0f} bpm ({summary['performance_trend']['avg_heart_rate']['change']:.0f} bpm、{summary['performance_trend']['avg_heart_rate']['change_pct']:.1f}%減少)
   - 同じペースでより低い心拍で走れるようになっています
   - これは有酸素能力の向上を示しています ✅

2. **ペースの変化（要注意）**
   - 平均ペース: {summary['performance_trend']['avg_pace']['first']//60:.0f}:{summary['performance_trend']['avg_pace']['first']%60:02.0f} → {summary['performance_trend']['avg_pace']['last']//60:.0f}:{summary['performance_trend']['avg_pace']['last']%60:02.0f} /km ({summary['performance_trend']['avg_pace']['change']:.0f}秒、{summary['performance_trend']['avg_pace']['change_pct']:.1f}%増加)
   - ペースがやや遅くなっていますが、心拍が下がっているため、意図的なイージーランの可能性があります
   - または疲労の蓄積の兆候かもしれません ⚠️

3. **心拍ゾーン分布の変化**
   - Zone 3の時間: {summary['hr_zone_trend']['zone_3_change']:+.0f}%
   - Zone 4の時間: {summary['hr_zone_trend']['zone_4_change']:+.0f}%
   - より低い心拍ゾーンでのトレーニングにシフトしています
   - これは有酸素ベースの強化に適した変化です ✅

**推奨:**
- 心拍効率の改善は非常にポジティブなサインです
- ペースの低下が意図的なものか確認してください
- 疲労が蓄積している場合は、リカバリー週を設けることを検討してください
- 次回のテストでは、同じペースでの心拍変化を追跡することをお勧めします
"""

print(interpretation)

## トークンコスト比較

### Before（従来の方法）:
- 5アクティビティの全時系列データ: 5 × 2700秒 × 2カラム × 平均10バイト = 約270KB
- トークンコスト: 約67,500トークン 😱

### After（新アーキテクチャ）:
1. プロファイル × 5: 約625トークン
2. MCP `export()` レスポンス × 5: 約125トークン
3. 要約JSON: 約150トークン
4. 合計: 約900トークン

**トークン削減率: 98.7%** 🎉🎉🎉

## まとめ

このノートブックでは、複数アクティビティ比較を通じて：

1. ✅ プロファイル → エクスポート → 正規化 → 比較の段階的アプローチ
2. ✅ 正規化時間軸による公平な比較
3. ✅ トレンド分析による長期的なパフォーマンス変化の把握
4. ✅ 98%以上のトークン削減を達成
5. ✅ LLMによる包括的なパフォーマンス評価

このパターンは、ペース比較、フォームメトリクス比較、季節別比較など、他の多数アクティビティ分析にも応用可能です。